# EfficientNet-VisionTransformer Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, LayerNormalization, MultiHeadAttention, Add, Dropout, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from datetime import datetime

# Parameters
num_classes = 6  # Ensure this matches the actual number of classes
image_size = (224, 224)
patch_size = 16
num_heads = 4
transformer_units = [128, 64]
dropout_rate = 0.1
batch_size = 32
epochs = 20

# EfficientNet Feature Extractor
def create_efficientnet_backbone(input_shape):
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False
    return base_model

# Vision Transformer Block
def transformer_block(inputs, projection_dim, num_heads):
    x1 = LayerNormalization(epsilon=1e-6)(inputs)
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
    x2 = Add()([attention_output, inputs])

    x3 = LayerNormalization(epsilon=1e-6)(x2)
    x3 = Dense(projection_dim, activation=tf.nn.gelu)(x3)
    x3 = Dropout(dropout_rate)(x3)
    x3 = Dense(projection_dim)(x3)
    return Add()([x3, x2])

# EfficientNet + Vision Transformer Model
def create_efficient_vit_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    efficientnet = create_efficientnet_backbone(input_shape)
    x = efficientnet(inputs)
    x = GlobalAveragePooling2D()(x)

    projection_dim = x.shape[-1]
    x = Reshape((1, projection_dim))(x)  # Reshape to match the attention mechanism requirements

    for _ in range(4):  # Transformer layers
        x = transformer_block(x, projection_dim, num_heads)

    x = LayerNormalization(epsilon=1e-6)(x)
    x = Flatten()(x)  # Flatten before the final Dense layer
    outputs = Dense(num_classes, activation="softmax")(x)  # Ensure num_classes is correct

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Custom Callback for Detailed Metrics
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, total_batches):
        super().__init__()
        self.total_batches = total_batches
        self.batch_counter = 1

    def on_epoch_begin(self, epoch, logs=None):
        self.batch_counter = 1  # Reset batch counter at the start of each epoch
        print(f"\nEpoch {epoch + 1}/{self.params['epochs']}")

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        accuracy = logs.get('accuracy', 0)
        loss = logs.get('loss', 0)
        precision = logs.get('custom_precision', 0)
        recall = logs.get('custom_recall', 0)
        f1 = logs.get('custom_f1', 0)
        specificity = logs.get('custom_specificity', 0)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"Batch {self.batch_counter}/{self.total_batches} ━━━━━━━━━━━━━━━━━━━━ {current_time}")
        print(f"Accuracy: {accuracy:.4f} - Precision: {precision:.4f} - Recall: {recall:.4f} - Specificity: {specificity:.4f} - F1: {f1:.4f} - Loss: {loss:.4f}\n")
        self.batch_counter += 1

# Compile Model
input_shape = (224, 224, 3)
model = create_efficient_vit_model(input_shape, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Data Loading and Preprocessing
csv_path = r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\FETAL_PLANES_DB_data.csv"
df = pd.read_csv(csv_path, delimiter=";")

# Shuffle the DataFrame for a random split
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Add .png extension to each image name
df["Image_name"] = df["Image_name"].apply(lambda x: f"{x}.png")

# Image data generator with train-validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="training"
)

val_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="validation"
)

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training with Metrics Callback
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[early_stopping, MetricsCallback(total_batches=len(train_gen))]
)

# Evaluate on Validation Data
val_loss, val_accuracy = model.evaluate(val_gen)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Calculate Metrics on Validation Set
val_preds = model.predict(val_gen)
val_labels = val_gen.classes

# Convert predictions to label format
val_preds = np.argmax(val_preds, axis=1)

precision = precision_score(val_labels, val_preds, average='weighted')
recall = recall_score(val_labels, val_preds, average='weighted')
f1 = f1_score(val_labels, val_preds, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Visualization of Training History
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


Found 9920 validated image filenames belonging to 6 classes.
Found 2480 validated image filenames belonging to 6 classes.

Epoch 1/20
Epoch 1/20


C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Batch 1/310 ━━━━━━━━━━━━━━━━━━━━ 15:48:15
Accuracy: 0.3125 - Precision: 0.0000 - Recall: 0.0000 - Specificity: 0.0000 - F1: 0.0000 - Loss: 2.4335

  1/310 ━━━━━━━━━━━━━━━━━━━━ 5:08:11 60s/step - accuracy: 0.3125 - loss: 2.4335Batch 2/310 ━━━━━━━━━━━━━━━━━━━━ 15:48:28
Accuracy: 0.3750 - Precision: 0.0000 - Recall: 0.0000 - Specificity: 0.0000 - F1: 0.0000 - Loss: 4.8172

  2/310 ━━━━━━━━━━━━━━━━━━━━ 1:02:49 12s/step - accuracy: 0.3438 - loss: 3.6253Batch 3/310 ━━━━━━━━━━━━━━━━━━━━ 15:48:35
Accuracy: 0.3125 - Precision: 0.0000 - Recall: 0.0000 - Specificity: 0.0000 - F1: 0.0000 - Loss: 6.0704

  3/310 ━━━━━━━━━━━━━━━━━━━━ 50:44 10s/step - accuracy: 0.3333 - loss: 4.4404  Batch 4/310 ━━━━━━━━━━━━━━━━━━━━ 15:48:47
Accuracy: 0.2891 - Precision: 0.0000 - Recall: 0.0000 - Specificity: 0.0000 - F1: 0.0000 - Loss: 7.1002

  4/310 ━━━━━━━━━━━━━━━━━━━━ 54:14 11s/step - accuracy: 0.3223 - loss: 5.1053Batch 5/310 ━━━━━━━━━━━━━━━━━━━━ 15:48:59
Accuracy: 0.2625 - Precision: 0.0000 - Recall: 0.0000 - 

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, LayerNormalization, MultiHeadAttention, Add, Dropout, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from datetime import datetime

# Parameters
num_classes = 6
image_size = (224, 224)
patch_size = 16
num_heads = 4
transformer_units = [128, 64]
dropout_rate = 0.1
batch_size = 32
epochs = 20

# EfficientNet Feature Extractor
def create_efficientnet_backbone(input_shape):
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False
    return base_model

# Vision Transformer Block
def transformer_block(inputs, projection_dim, num_heads):
    x1 = LayerNormalization(epsilon=1e-6)(inputs)
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
    x2 = Add()([attention_output, inputs])

    x3 = LayerNormalization(epsilon=1e-6)(x2)
    x3 = Dense(projection_dim, activation=tf.nn.gelu)(x3)
    x3 = Dropout(dropout_rate)(x3)
    x3 = Dense(projection_dim)(x3)
    return Add()([x3, x2])

# EfficientNet + Vision Transformer Model
def create_efficient_vit_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    efficientnet = create_efficientnet_backbone(input_shape)
    x = efficientnet(inputs)
    x = GlobalAveragePooling2D()(x)
    projection_dim = x.shape[-1]
    x = Reshape((1, projection_dim))(x)

    for _ in range(4):
        x = transformer_block(x, projection_dim, num_heads)

    x = LayerNormalization(epsilon=1e-6)(x)
    x = Flatten()(x)
    outputs = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Custom Callback for Multi-Class Metrics
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_data, total_batches):
        super().__init__()
        self.val_data = val_data
        self.total_batches = total_batches
        self.batch_counter = 1
        self.precision_metric = tf.keras.metrics.Precision()
        self.recall_metric = tf.keras.metrics.Recall()
        self.specificity_metric = tf.keras.metrics.SpecificityAtSensitivity(0.5)

    def on_epoch_begin(self, epoch, logs=None):
        self.batch_counter = 1
        print(f"\nEpoch {epoch + 1}/{self.params['epochs']}")

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        accuracy = logs.get('accuracy', 0)
        loss = logs.get('loss', 0)
        
        # Fetch validation data
        x_val, y_true = next(iter(self.val_data))
        y_pred = self.model.predict(x_val, verbose=0)
        y_pred_class = np.argmax(y_pred, axis=1)
        
        # One-hot encoding for calculations
        y_true_one_hot = tf.one_hot(y_true, depth=num_classes)
        y_pred_one_hot = tf.one_hot(y_pred_class, depth=num_classes)
        
        # Update multi-class precision and recall
        self.precision_metric.update_state(y_true_one_hot, y_pred_one_hot)
        self.recall_metric.update_state(y_true_one_hot, y_pred_one_hot)
        
        # Calculate F1 Score manually
        precision = self.precision_metric.result().numpy()
        recall = self.recall_metric.result().numpy()
        f1 = 2 * (precision * recall) / (precision + recall + 1e-7)

        # Specificity calculation: TN / (TN + FP) for each class
        tn = tf.reduce_sum((1 - y_true_one_hot) * (1 - y_pred_one_hot))
        fp = tf.reduce_sum((1 - y_true_one_hot) * y_pred_one_hot)
        specificity = tn / (tn + fp + 1e-7)
        
        # Display metrics
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"Batch {self.batch_counter}/{self.total_batches} ━━━━━━━━━━━━━━━━━━━━ {current_time}")
        print(f"Accuracy: {accuracy:.4f} - Precision: {precision:.4f} - Recall: {recall:.4f} - Specificity: {specificity:.4f} - F1: {f1:.4f} - Loss: {loss:.4f}\n")
        self.batch_counter += 1

# Compile Model
input_shape = (224, 224, 3)
model = create_efficient_vit_model(input_shape, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Data Loading and Preprocessing
csv_path = r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\FETAL_PLANES_DB_data.csv"
df = pd.read_csv(csv_path, delimiter=";")
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df["Image_name"] = df["Image_name"].apply(lambda x: f"{x}.png")

# Image data generator with train-validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="training"
)

val_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="validation"
)

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training with Metrics Callback
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[early_stopping, MetricsCallback(val_data=val_gen, total_batches=len(train_gen))],
    verbose=0
)

# Evaluate on Validation Data
val_loss, val_accuracy = model.evaluate(val_gen)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Visualization of Training History
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


Found 9920 validated image filenames belonging to 6 classes.
Found 2480 validated image filenames belonging to 6 classes.

Epoch 1/20


C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (32, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid i

Batch 1/310 ━━━━━━━━━━━━━━━━━━━━ 15:19:45
Accuracy: 0.0625 - Precision: 0.2188 - Recall: 0.2188 - Specificity: 0.8438 - F1: 0.2187 - Loss: 3.0772

Batch 2/310 ━━━━━━━━━━━━━━━━━━━━ 15:20:01
Accuracy: 0.1719 - Precision: 0.2188 - Recall: 0.2188 - Specificity: 0.8438 - F1: 0.2187 - Loss: 4.3618

Batch 3/310 ━━━━━━━━━━━━━━━━━━━━ 15:20:10
Accuracy: 0.1875 - Precision: 0.1979 - Recall: 0.1979 - Specificity: 0.8313 - F1: 0.1979 - Loss: 5.0726

Batch 4/310 ━━━━━━━━━━━━━━━━━━━━ 15:20:20
Accuracy: 0.1641 - Precision: 0.1641 - Recall: 0.1641 - Specificity: 0.8125 - F1: 0.1641 - Loss: 6.5093



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, LayerNormalization, MultiHeadAttention, Add, Dropout, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from datetime import datetime

# Parameters
num_classes = 6  # Ensure this matches the actual number of classes
image_size = (224, 224)
patch_size = 16
num_heads = 4
transformer_units = [128, 64]
dropout_rate = 0.1
batch_size = 32
epochs = 20

# EfficientNet Feature Extractor
def create_efficientnet_backbone(input_shape):
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False
    return base_model

# Vision Transformer Block
def transformer_block(inputs, projection_dim, num_heads):
    x1 = LayerNormalization(epsilon=1e-6)(inputs)
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
    x2 = Add()([attention_output, inputs])

    x3 = LayerNormalization(epsilon=1e-6)(x2)
    x3 = Dense(projection_dim, activation=tf.nn.gelu)(x3)
    x3 = Dropout(dropout_rate)(x3)
    x3 = Dense(projection_dim)(x3)
    return Add()([x3, x2])

# EfficientNet + Vision Transformer Model
def create_efficient_vit_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    efficientnet = create_efficientnet_backbone(input_shape)
    x = efficientnet(inputs)
    x = GlobalAveragePooling2D()(x)

    projection_dim = x.shape[-1]
    x = Reshape((1, projection_dim))(x)  # Reshape to match the attention mechanism requirements

    for _ in range(4):  # Transformer layers
        x = transformer_block(x, projection_dim, num_heads)

    x = LayerNormalization(epsilon=1e-6)(x)
    x = Flatten()(x)  # Flatten before the final Dense layer
    outputs = Dense(num_classes, activation="softmax")(x)  # Ensure num_classes is correct

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Custom Callback for Detailed Metrics
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, validation_data, total_batches):
        super().__init__()
        self.validation_data = validation_data
        self.total_batches = total_batches
        self.batch_counter = 1

    def on_epoch_begin(self, epoch, logs=None):
        self.batch_counter = 1  # Reset batch counter at the start of each epoch
        print(f"\nEpoch {epoch + 1}/{self.params['epochs']}")

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        accuracy = logs.get('accuracy', 0)
        loss = logs.get('loss', 0)

        # Compute precision, recall, F1, and specificity for the batch
        val_data, val_labels = self.validation_data
        val_preds = np.argmax(self.model.predict(val_data), axis=1)

        precision = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
        recall = recall_score(val_labels, val_preds, average='weighted', zero_division=0)
        f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)

        # Compute specificity
        cm = confusion_matrix(val_labels, val_preds)
        tn = np.diag(cm)
        specificity = (tn / (tn + cm.sum(axis=1) - tn)).mean()

        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"Batch {self.batch_counter}/{self.total_batches} ━━━━━━━━━━━━━━━━━━━━ {current_time}")
        print(f"Accuracy: {accuracy:.4f} - Precision: {precision:.4f} - Recall: {recall:.4f} - Specificity: {specificity:.4f} - F1: {f1:.4f} - Loss: {loss:.4f}\n")
        self.batch_counter += 1

# Compile Model
input_shape = (224, 224, 3)
model = create_efficient_vit_model(input_shape, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Data Loading and Preprocessing
csv_path = r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\FETAL_PLANES_DB_data.csv"
df = pd.read_csv(csv_path, delimiter=";")

# Shuffle the DataFrame for a random split
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Add .png extension to each image name
df["Image_name"] = df["Image_name"].apply(lambda x: f"{x}.png")

# Image data generator with train-validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="training"
)

val_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="validation",
    shuffle=False
)

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Prepare validation data for metrics callback
val_data, val_labels = next(val_gen)
validation_data = (val_data, val_labels)

# Training with Metrics Callback
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[early_stopping, MetricsCallback(validation_data, total_batches=len(train_gen))],
    verbose=0  # Set verbose to 0 to avoid duplicate output
)

# Evaluate on Validation Data
val_loss, val_accuracy = model.evaluate(val_gen)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Calculate Metrics on Validation Set
val_preds = model.predict(val_gen)
val_labels = val_gen.classes

# Convert predictions to label format
val_preds = np.argmax(val_preds, axis=1)

precision = precision_score(val_labels, val_preds, average='weighted')
recall = recall_score(val_labels, val_preds, average='weighted')
f1 = f1_score(val_labels, val_preds, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Visualization of Training History
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


Found 9920 validated image filenames belonging to 6 classes.
Found 2480 validated image filenames belonging to 6 classes.

Epoch 1/20


C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (32, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid i

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
Batch 1/310 ━━━━━━━━━━━━━━━━━━━━ 16:08:58
Accuracy: 0.1562 - Precision: 0.0625 - Recall: 0.2500 - Specificity: 0.1667 - F1: 0.1000 - Loss: 2.4024

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Batch 2/310 ━━━━━━━━━━━━━━━━━━━━ 16:09:13
Accuracy: 0.2344 - Precision: 0.0625 - Recall: 0.2500 - Specificity: 0.1667 - F1: 0.1000 - Loss: 3.8410

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Batch 3/310 ━━━━━━━━━━━━━━━━━━━━ 16:09:23
Accuracy: 0.2083 - Precision: 0.0244 - Recall: 0.1562 - Specificity: 0.1667 - F1: 0.0422 - Loss: 6.1864

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Batch 4/310 ━━━━━━━━━━━━━━━━━━━━ 16:09:33
Accuracy: 0.1719 - Precision: 0.0791 - Recall: 0.2812 - Specificity: 0.1667 - F1: 0.1235 - Loss: 7.3789

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Batch 5/310 ━━━━━━━━━━━━━━━━━━━━ 16:09:48
Accuracy: 0.1875 - Precision: 0.0010 - Recall: 0.0312 - Specificity: 0.1667 - F1: 0.0019 - Loss: 7.5390

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Batch 6/310 ━━━━━━━━━━━━━━━━━━━━ 16:10:25
Accur

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, LayerNormalization, MultiHeadAttention, Add, Dropout, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from datetime import datetime

# Parameters
num_classes = 6  # Ensure this matches the actual number of classes
image_size = (224, 224)
patch_size = 16
num_heads = 4
transformer_units = [128, 64]
dropout_rate = 0.1
batch_size = 32
epochs = 20

# EfficientNet Feature Extractor
def create_efficientnet_backbone(input_shape):
    base_model = EfficientNetB0(include_top=False, weights="imagenet", input_shape=input_shape)
    base_model.trainable = False
    return base_model

# Vision Transformer Block
def transformer_block(inputs, projection_dim, num_heads):
    x1 = LayerNormalization(epsilon=1e-6)(inputs)
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
    x2 = Add()([attention_output, inputs])
    x3 = LayerNormalization(epsilon=1e-6)(x2)
    x3 = Dense(projection_dim, activation=tf.nn.gelu)(x3)
    x3 = Dropout(dropout_rate)(x3)
    x3 = Dense(projection_dim)(x3)
    return Add()([x3, x2])

# EfficientNet + Vision Transformer Model
def create_efficient_vit_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    efficientnet = create_efficientnet_backbone(input_shape)
    x = efficientnet(inputs)
    x = GlobalAveragePooling2D()(x)
    projection_dim = x.shape[-1]
    x = Reshape((1, projection_dim))(x)  # Reshape to match the attention mechanism requirements
    for _ in range(4):  # Transformer layers
        x = transformer_block(x, projection_dim, num_heads)
    x = LayerNormalization(epsilon=1e-6)(x)
    x = Flatten()(x)  # Flatten before the final Dense layer
    outputs = Dense(num_classes, activation="softmax")(x)  # Ensure num_classes is correct
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Custom Callback for Detailed Metrics
class MetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, total_batches):
        super().__init__()
        self.total_batches = total_batches

    def on_epoch_begin(self, epoch, logs=None):
        print(f"\nEpoch {epoch + 1}/{self.params['epochs']}")

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        accuracy = logs.get('accuracy', 0)
        loss = logs.get('loss', 0)
        current_time = datetime.now().strftime("%H:%M:%S")
        print(f"Batch {batch+1}/{self.total_batches} ━━━━━━━━━━━━━━━━━━━━ {current_time}")
        print(f"Accuracy: {accuracy:.4f} - Loss: {loss:.4f}\n")

# Compile Model
input_shape = (224, 224, 3)
model = create_efficient_vit_model(input_shape, num_classes)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# Data Loading and Preprocessing
csv_path = r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\FETAL_PLANES_DB_data.csv"
df = pd.read_csv(csv_path, delimiter=";")

# Shuffle the DataFrame for a random split
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Add .png extension to each image name
df["Image_name"] = df["Image_name"].apply(lambda x: f"{x}.png")

# Image data generator with train-validation split
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="training"
)

val_gen = datagen.flow_from_dataframe(
    dataframe=df,
    directory=r"C:\Users\Jaber\OneDrive - University of Florida\Educational\GitHub\Ultrasound_Fetal\Data\Images",
    x_col="Image_name",
    y_col="Plane",
    target_size=image_size,
    class_mode="sparse",
    batch_size=batch_size,
    subset="validation"
)

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training with Metrics Callback
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,
    callbacks=[early_stopping, MetricsCallback(total_batches=len(train_gen))],
    verbose=0  # Set verbose to 0 to avoid TensorFlow's default logging per batch
)

# Evaluate on Validation Data
val_loss, val_accuracy = model.evaluate(val_gen)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

# Visualization of Training History
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


Found 9920 validated image filenames belonging to 6 classes.
Found 2480 validated image filenames belonging to 6 classes.

Epoch 1/20


C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\ops\nn.py:545: UserWarning: You are using a softmax over axis 3 of a tensor of shape (None, 4, 1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
C:\ProgramData\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Batch 1/310 ━━━━━━━━━━━━━━━━━━━━ 16:13:19
Accuracy: 0.2188 - Loss: 2.5990

Batch 2/310 ━━━━━━━━━━━━━━━━━━━━ 16:13:30
Accuracy: 0.2031 - Loss: 6.6319

Batch 3/310 ━━━━━━━━━━━━━━━━━━━━ 16:13:36
Accuracy: 0.1771 - Loss: 7.0872

Batch 4/310 ━━━━━━━━━━━━━━━━━━━━ 16:13:43
Accuracy: 0.1875 - Loss: 7.4784

Batch 5/310 ━━━━━━━━━━━━━━━━━━━━ 16:13:49
Accuracy: 0.2125 - Loss: 6.6678

Batch 6/310 ━━━━━━━━━━━━━━━━━━━━ 16:13:55
Accuracy: 0.2708 - Loss: 5.9960

Batch 7/310 ━━━━━━━━━━━━━━━━━━━━ 16:14:02
Accuracy: 0.2723 - Loss: 5.7551

Batch 8/310 ━━━━━━━━━━━━━━━━━━━━ 16:14:10
Accuracy: 0.2500 - Loss: 5.5692

Batch 9/310 ━━━━━━━━━━━━━━━━━━━━ 16:14:17
Accuracy: 0.2465 - Loss: 5.4121

Batch 10/310 ━━━━━━━━━━━━━━━━━━━━ 16:14:24
Accuracy: 0.2281 - Loss: 5.2516

Batch 11/310 ━━━━━━━━━━━━━━━━━━━━ 16:14:31
Accuracy: 0.2386 - Loss: 5.0206

Batch 12/310 ━━━━━━━━━━━━━━━━━━━━ 16:14:38
Accuracy: 0.2448 - Loss: 4.8028

Batch 13/310 ━━━━━━━━━━━━━━━━━━━━ 16:14:45
Accuracy: 0.2476 - Loss: 4.6286

Batch 14/310 ━━━━━━━━